In [1]:
from bs4 import BeautifulSoup as bs
import time, requests, json,re
from IPython.display import clear_output

In [2]:
def get_web_page(url):
    resp=requests.get(
        url=url,
        cookies={'over18':'1'})
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        resp.encoding = 'utf-8'
        return resp.text

In [3]:
def get_ip(dom):
    pattern = '來自: \d+\.\d+\.\d+\.\d+'
    match = re.search(pattern, dom)
    if match:
        return match.group(0).replace('來自: ', '')
    else:
        return None

In [4]:
def get_country(ip):
    if ip:
        url = 'http://api.ipstack.com/{}?access_key={}'.format(ip,api_key)
        print(url)
        data = requests.get(url).json()
        country_name = data['country_name'] if data['country_name'] else None
        return country_name
    return None

In [5]:
def get_articles(dom, date):
    soup = bs(dom,'html5lib')
#     print(soup)
    #Get previous page link
    paging_div = soup.find('div','btn-group btn-group-paging')
#     print(paging_div)
    prev_url = paging_div.find_all('a')[1]['href']
    
    articles = []
    divs = soup.find_all('div','r-ent')
    push_count = 0
    for d in divs:
        if d.find('div', 'date').text.strip() == date:
            clear_output(wait=True)
            print(d.find('div', 'date').text.strip(), date)
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)
                except ValueError:
                    # if transfer failure maybe is '爆', 'x1', 'x2'
                    # if don`t do anything push_count keep 0
                    if push_str == '爆':
                        push_count == 99
                    elif push_str.startswith('X'):
                        push_count = -10
            if d.find('a'): #Have hyperLink means paragraph exist else delete
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title' : title,
                    'href' : href,
                    'push_count' : push_count,
                    'author' : author
                })          
                print(articles[0]['title'],articles[0]['href'])
    return articles, prev_url

In [6]:
api_key = '0f446af799762c27cd408fa9919e041e'
ppt_url = 'https://www.ptt.cc'

In [7]:
print('get today`s article...')
current_page = get_web_page(ppt_url +'/bbs/Gossiping/index.html')
if current_page:
    articles = []
    today = time.strftime('%m/%d').lstrip('0')
#     Today`s date
    current_articles, prev_url = get_articles(current_page, today)
#     today`s article in today`s date
    while current_articles:
        articles += current_articles
        current_page = get_web_page(ppt_url + prev_url)
        current_articles, prev_url = get_articles(current_page,today)
    print('Total %d article' % (len(articles)))
    
    print('Taken recent 100 articles`s IP')
    country_to_count = dict()
    for article in articles[:100]:
        clear_output(wait=True)
        print('Search for IP:', article['title'])
        page = get_web_page(ppt_url + article['href'])
        if page:
            ip = get_ip(page)
            country = get_country(ip)
            if country in country_to_count.keys():
                country_to_count[country] += 1
            else:
                country_to_count[country] = 1
    print('Each country`s IP')
    for k, v in country_to_count.items():
        print(k,v)
        

Search for IP: [新聞] 韓國瑜隨身攜茶葉 營養師：含兒茶素可提
http://api.ipstack.com/124.217.249.24?access_key=0f446af799762c27cd408fa9919e041e
Each country`s IP
Taiwan 93
France 2
Japan 1
United States 2
South Korea 1
Malaysia 1
